In [1]:
# 批量归一化 — 从0开始
# http://zh.gluon.ai/chapter_convolutional-neural-networks/batch-norm-scratch.html

# 简化的批量归一化层
from mxnet import nd
def pure_batch_norm(X, gamma, beta, eps=1e-5):
    assert len(X.shape) in (2, 4)
    # either 2 or 4, why? 2 is for fc. 4 is for 2d convnet.
    # TODO: what about 1d or 3d convnet?
    # where is gamma, and beta come from? these are learnable
    
    # 全连接: batch_size x feature
    if len(X.shape) == 2:
        # 每个输入维度在样本上的平均和方差
        mean = X.mean(axis=0)
        variance = ((X - mean)**2).mean(axis=0)
    # 2D卷积: batch_size x channel x height x width
    else:
        # 对每个通道算均值和方差，需要保持4D形状使得可以正确地广播
        mean = X.mean(axis=(0, 2, 3), keepdims=True)
        variance = ((X-mean)**2).mean(axis=(0,2,3), keepdims=True)
    
    # 均一化
    X_hat = (X - mean) / nd.sqrt(variance + eps)
    # 拉升和偏移
    return gamma.reshape(mean.shape) * X_hat + beta.reshape(mean.shape)

/home/ly/anaconda3/envs/learning/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
A = nd.arange(6).reshape((3,2))
A


[[0. 1.]
 [2. 3.]
 [4. 5.]]
<NDArray 3x2 @cpu(0)>

In [4]:
pure_batch_norm(A, gamma=nd.array([1,1]), beta=nd.array([0,0]))


[[-1.2247427 -1.2247427]
 [ 0.         0.       ]
 [ 1.2247427  1.2247427]]
<NDArray 3x2 @cpu(0)>

In [5]:
B = nd.arange(18).reshape((1,2,3,3))
B


[[[[ 0.  1.  2.]
   [ 3.  4.  5.]
   [ 6.  7.  8.]]

  [[ 9. 10. 11.]
   [12. 13. 14.]
   [15. 16. 17.]]]]
<NDArray 1x2x3x3 @cpu(0)>

In [6]:
pure_batch_norm(B, gamma=nd.array([1,1]), beta=nd.array([0,0]))


[[[[-1.5491922  -1.1618942  -0.7745961 ]
   [-0.38729805  0.          0.38729805]
   [ 0.7745961   1.1618942   1.5491922 ]]

  [[-1.5491922  -1.1618942  -0.7745961 ]
   [-0.38729805  0.          0.38729805]
   [ 0.7745961   1.1618942   1.5491922 ]]]]
<NDArray 1x2x3x3 @cpu(0)>

In [51]:
# 批量归一化层
def batch_norm(X, gamma, beta, is_training,
               moving_mean, moving_variance,
              eps=1e-5, moving_momentum=0.9):
    assert len(X.shape) in (2, 4)
    # 全连接: batch_size x feature
    if len(X.shape) == 2:
        # 每个输入维度在样本上的平均和方差
        mean = X.mean(axis=0)
        variance = ((X - mean)**2).mean(axis=0)
    # 2D卷积: batch_size x channel x height x width
    else:
        # 对每个通道算均值和方差，需要保持4D形状使得可以正确的广播
        mean = X.mean(axis=(0,2,3), keepdims=True)
        variance = ((X - mean)**2).mean(axis=(0,2,3), keepdims=True)
        # 变形使得可以正确的广播
        moving_mean = moving_mean.reshape(mean.shape)
        moving_variance = moving_variance.reshape(mean.shape)
    
    # 均一化
    if is_training:
        X_hat = (X - mean) / nd.sqrt(variance + eps)
        #!!! 更新全局的均值和方差
        moving_mean[:] = moving_momentum * moving_mean
        + (1.0 - moving_momentum) * mean
        moving_variance[:] = moving_momentum * moving_variance
        + (1.0 - moving_momentum) * variance
    else:
        #!!! 测试阶段使用全局的均值和方差
        X_hat = (X - moving_mean) / nd.sqrt(moving_variance + eps)
#         print('moving mean:', moving_mean)
    
    # 拉升和偏移
    return gamma.reshape(mean.shape) * X_hat + beta.reshape(mean.shape)

In [9]:
# 定义模型
import sys
sys.path.append('..')
import utils
ctx = utils.try_gpu()
ctx

gpu(0)

In [22]:
from mxnet import autograd
from mxnet import gluon

batch_size = 256
def transform(data, label):
#     print(data.shape)
#     print(label.shape)
# change data from batch x height x width x channel
# to batch x channel x height x width
#     return nd.transpose(data.astype('float32'), (0,3,1,2))/255, label.astype('float32')
    return nd.transpose(data.astype('float32'), (2,0,1))/255, label.astype('float32')
#     return data.astype('float32') / 255, label.astype('float32')
mnist_train = gluon.data.vision.FashionMNIST(train=True, transform=transform)
mnist_test = gluon.data.vision.FashionMNIST(train=False, transform=transform)
train_data = gluon.data.DataLoader(mnist_train, batch_size, shuffle=True)
test_data = gluon.data.DataLoader(mnist_test, batch_size, shuffle=False)

In [48]:
weight_scale = 0.01

# 输出通道 = 20, 卷积核 = (5,5)
c1 = 20
W1 = nd.random.normal(shape=(c1, 1, 5, 5), scale=weight_scale, ctx=ctx)
b1 = nd.zeros(c1, ctx=ctx)

# 第1层批量归一化
gamma1 = nd.random.normal(shape=c1, scale=weight_scale, ctx=ctx)
beta1 = nd.random.normal(shape=c1, scale=weight_scale, ctx=ctx)
moving_mean1 = nd.zeros(c1, ctx=ctx)
moving_variance1 = nd.ones(c1, ctx=ctx)

# 输出通道 = 50, 卷积核 = (3,3)
c2 = 50
W2 = nd.random_normal(shape=(c2, c1, 3, 3), scale=weight_scale, ctx=ctx)
b2 = nd.zeros(c2, ctx=ctx)

# 第2层批量归一化
gamma2 = nd.random.normal(shape=c2, scale=weight_scale, ctx=ctx)
beta2 = nd.random.normal(shape=c2, scale=weight_scale, ctx=ctx)
moving_mean2 = nd.zeros(c2, ctx=ctx)
moving_variance2 = nd.ones(c2, ctx=ctx)

# 输出维度 = 128
o3 = 128
# why 1250?
# 从conv (kernel, stride), pooling(kernel, stride), flatten计算出来的
W3 = nd.random.normal(shape=(1250, o3), scale=weight_scale, ctx=ctx)
b3 = nd.zeros(o3, ctx=ctx)

# 输出维度 = 10
o4 = 10
W4 = nd.random_normal(shape=(o3, o4), scale=weight_scale, ctx=ctx)
b4 = nd.zeros(o4, ctx=ctx)

# 注意这里moving_*是不需要更新的
params = [W1, b1, gamma1, beta1,
         W2, b2, gamma2, beta2,
         W3, b3, W4, b4]

for param in params:
    param.attach_grad()

In [49]:
def net(X, is_training=False, verbose=False):
    X = X.as_in_context(W1.context)
    # 第一层卷积
    h1_conv = nd.Convolution(
        data=X, weight=W1, bias=b1, kernel=W1.shape[2:], num_filter=c1
    )
    ### 添加了批量归一化层
    h1_bn = batch_norm(h1_conv, gamma1, beta1, is_training,
                      moving_mean1, moving_variance1)
    h1_activation = nd.relu(h1_bn)
    h1 = nd.Pooling(data=h1_activation, pool_type='max',
                   kernel=(2, 2), stride=(2,2))
    # 第二层卷积
    h2_conv = nd.Convolution(
        data=h1, weight=W2, bias=b2, kernel=W2.shape[2:], num_filter=c2
    )
    h2_bn = batch_norm(h2_conv, gamma2, beta2, is_training,
                      moving_mean2, moving_variance2)
    h2_activation = nd.relu(h2_bn)
    h2 = nd.Pooling(data=h2_activation, pool_type='max',
                   kernel=(2,2), stride=(2,2))
    h2 = nd.flatten(h2)
    # 第一层全连接
    h3_linear = nd.dot(h2, W3) + b3
    h3 = nd.relu(h3_linear)
    # 第二层全连接
    h4_linear = nd.dot(h3, W4) + b4
    if verbose:
        print('X:', X.shape)
        print('1st conv block:', h1.shape)
        print('2nd conv block:', h2.shape)
        print('1st dense:', h3.shape)
        print('2nd dense:', h4_linear.shape)
        print('output:', h4_linear)
    return h4_linear

In [52]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

learning_rate = 0.2

for epoch in range(10):
    train_loss = 0.
    train_acc = 0.
    for data, label in train_data:
        label = label.as_in_context(ctx)
        with autograd.record():
            output = net(data, is_training=True, verbose=False)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        utils.SGD(params, learning_rate / batch_size)
        
        train_loss += nd.mean(loss).asscalar()
        train_acc += utils.accuracy(output, label)
        
    test_acc = utils.evaluate_accuracy(test_data, net, ctx)
    print("Epoch %d. Loss: %f, Train acc %f, Test acc %f" % (
            epoch, train_loss/len(train_data), train_acc/len(train_data), test_acc))

Epoch 0. Loss: 0.525445, Train acc 0.797435, Test acc 0.790430
Epoch 1. Loss: 0.411807, Train acc 0.845761, Test acc 0.819434
Epoch 2. Loss: 0.353961, Train acc 0.868163, Test acc 0.832617
Epoch 3. Loss: 0.323986, Train acc 0.880075, Test acc 0.829883
Epoch 4. Loss: 0.299802, Train acc 0.889644, Test acc 0.843164
Epoch 5. Loss: 0.284036, Train acc 0.894166, Test acc 0.863086
Epoch 6. Loss: 0.271367, Train acc 0.898997, Test acc 0.806445
Epoch 7. Loss: 0.258008, Train acc 0.904028, Test acc 0.855566
Epoch 8. Loss: 0.246977, Train acc 0.908793, Test acc 0.860352
Epoch 9. Loss: 0.235529, Train acc 0.913259, Test acc 0.843262
